# West Nile Virus Predicition
## <u>Predicting West Nile Virus in Mosquitos across the City of Chicago</u>


## 1. Introduction <a id='Introduction'></a>
### 1.1 What is the West Nile Virus?
West Nile fever is an infection by the West Nile virus (WNV), which is typically spread by mosquitoes. Mosquitoes become infected when they feed on infected birds which carry the disease. Upon feeding on these birds, the mosquitoes become infected as well, and pass the disease on to humans when they bite. Cases of WNV generally occur during mosquito season, which starts in the summer and continues through fall.

### 1.2 Problem Statement
There are two issues we seek to address in this project:

Firstly, we aim to build a model capable of predicting the outbreak of West Nile Virus in the Chicago locale. We will not be looking at the mosquito-to-human transmission of WNV, as this is a function of complex environmental and health factors that is out of the scope of this project. Rather, we will be predicting the appearance of WNV in the mosquitos themselves (i.e. detecting the presence of WNV in mosquitos).

By predicting the appearance of WNV-infected mosquitos, our analyses should lend itself to further studies on the interaction between infected mosquitos and human/environmental variables. Furthermore, through predicting the outbreak of infected mosquitos in specific geographic regions, we would be able to support a local government's attempts to control mosquito populations via spraying efforts by pointing them towards high risk areas, whilst lowering costs associated with wide scale mosquito spraying.

Secondly, we want the model to be interpretable, so as to be able to make recommendations for local governments on how to prepare for potential WNV outbreaks, and how to prevent WNV altogether by taking precautions. For example, there is extensive literature on how mosquito activity is a function of temperature, humidity, and precipitation. We would like to explore if there are features other than this that contribute to mosquitos becoming WNV vectors. 

### 1.3 Modelling Approach and Strategy
The presence of WNV is a function of three things:
> 1. <b>Mosquito activity</b>: Mosquitos are more active during hot and humid periods. This increased activity leads to increased transmission of the virus between bird, mosquito, and human, as feeding frequency increases. We will, therefore, build several weather features into our model. This includes point humidity, temperature, precipation, as well as rolling averages of these factors. Length of day (i.e. period between sunrise and sundown) is also hypothesized to be a predictor of mosquito activity, due to the increased activity periods.

> 2. <b>Mosquito breeding</b>: Mosquitos breed in stagnant, standing fresh water. Their eggs can survive in stasis for more than 10 years, and will only hatch when the conditions are right. Mosquitoes begin to hatch, breed, and attack when the weather is at a consistent 50 degrees Fahrenheit (10 degrees Celsius), but mosquito eggs held at 22 and 27°C had the highest overall mean hatching count, as described in <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2705337/">this study.</a>. This also implies that prior years' mosquito populations could lead to future outbreaks, particularly when there are long periods of drought or cold weather.


> 3. <b>Presence of infected birds</b>: The number of mosquitos alone is a not an accurate predictor of WNV, as the mosquito needs to feed on an infected bird before becoming a disease vector. In the United States, WNV mosquito vectors feed preferentially on members of the Corvidae and thrush family. Among the preferred species within these families are the <a href = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1636093'>American crow, corvids, and the American robin. </a> Areas with higher populations of these birds are hypothesized to be at greater risk of WNV, though the diverse habitats of these species means that we may not have sufficient data on hand to make deeper analyses on the interaction effect between bird and mosquito.

Our feature selection strategy combines these three elements.
Because the proportion of WNV infected mosquitos to non-infected mosquitos is fairly low, our data set is heavily imbalanced and skewed towards non-WNV carrying mosquitos (i.e. WnvPresent = 0). Two strategies will be attempted to address this: 1) The generation of synthetic samples (SMOTE), and 2) The use of models that can handle imbalanced classes (Random Forests).

A variety of models will be generated, and these models will be combined via a stacking strategy utilizing the mlxtend library - an ensemble-learning meta-classifier capable of ensembling models using different feature sets. By ensembling these models, we will be leveraging on the 'wisdom of the crowd' phenemonon to make our predictions as each model is expected to have its own strengths and weaknesses in making predictions. Ensembling should therefore cancel out the various models' weaknesses and lend to a better overall model.

For evaluating our model, we will use two metrics: AUC-ROC score, and Recall. The AUC-ROC score gives us a good idea of how well the model performances in terms of its ability to distinguish positive and the negative values, whilst the Recall score is important as it represents our model's ability to correctly predict the apperance of WNV infected mosquitos.

### 1.4 Structure of Project

This project is split into 6 notebooks:

1) Introduction and Data Cleaning

2) Exploratory Data Analysis

3) Baseline Model and Generalized Additive Models

4) CART Models

5) Clustered CART Model

6) Summary and Conclusions

## 2. Table of Contents  <a id='Table of Contents'></a>

- <a href='#Introduction'>1. Introduction</a>
    - <a href='#Introduction'>1.1 What is the West Nile Virus</a>
    - <a href='#Introduction'>1.2 Problem Statement</a>
    - <a href='#Introduction'>1.3 Modelling Strategy</a>


- <a href='#Table of Contents'>2. Table of Contents</a>


- <a href='#Data Imports'>3. Library Imports</a>


- <a href='#Data Overview'>4. Data Imports and Overview</a>
    - <a href='#Evaluating Shape'>4.1 Evaluating Shape, Missingness, and Duplicates</a>    
    - <a href='#Investigating Duplicates'>4.2 Investigating Duplicates</a>
    - <a href='#Analysis of Datasets'>4.3 Analysis of Datasets</a>
    - <a href='#Data Dictionary'>4.4. Data Dictionary</a>             
    
  
- <a href='#Model 1'>5. Feature Engineering - Model 1</a>    
    - <a href='#Model 1'>5.1 Correcting Date Time Data</a>  
    - <a href='#Grouping Mosquitos'>5.2 Grouping Mosquitos</a>  
    - <a href='#Diff Weather'>5.3 Cleaning and Evaluating Differences in Weather Stations</a>  
    - <a href='#Clean Train'>5.4 Cleaning Train Data</a>  
    - <a href='#Eval Geog'>5.5 Evaluating Geography</a>  
    - <a href='#Dummy Var'>5.6 Dummying Variables</a>      
    
    
- <a href='#Model 2'>6. Feature Engineering - Model 2</a>
    - <a href='#clean weather 2'>6.1 Dummying Variables</a>   
    - <a href='#clean train 2'>6.2 Cleaning Train and Test Data</a>   
    - <a href='#clean spray 2'>6.3 Cleaning Spray Data</a>   

## 3. Library Imports  <a id='Data Imports'></a>

In [ ]:
#General Imports
import pandas as pd
import numpy as np
import time
import random
import math
from collections import namedtuple, Counter
import scipy.stats as stats
from scipy.stats import norm
import datetime
from haversine import haversine, Unit
from math import radians, sin, cos, asin, sqrt

#Plotting/Graphs
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits import mplot3d
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm

%matplotlib inline

#Modelling Imports
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, roc_auc_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import stop_words
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.cluster import KMeans, DBSCAN, MeanShift
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster
from scipy.spatial.distance import pdist
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

#Lets you view all columns in a dataframe when there are a LARGE number of columns
pd.set_option('display.max_columns', None)
#Fix for pandas truncating long strings (e.g. urls)
#If you don't run this, pandas will cut short your long urls and this breaks code that require urls
pd.set_option("display.max_colwidth", 10000)



In [2]:
#Fun stuff
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from ipywidgets import *

### 3.1 Cleaning and Encoding Functions

In [3]:
#Removes duplicates and prints out number of duplicates removed
def remove_duplicates(my_df):
    before_d = len(my_df)
    my_df.drop_duplicates(inplace=True)
    after_d = len(my_df)
    print(str(before_d-after_d) + " duplicates were removed!")

In [4]:
"""
Spring runs from March 1 to May 31;
Summer runs from June 1 to August 31;
Fall (autumn) runs from September 1 to November 30; and.
Winter runs from December 1 to February 28 (February 29 in a leap year).
"""

def get_season(date):
    """
    convert date to month and day as integer (md), e.g. 4/21 = 421, 11/17 = 1117, etc.
    """
    m = date.month * 100
    d = date.day
    md = m + d

    if ((md >= 301) and (md <= 531)):
        s = 'spring'
    elif ((md > 531) and (md < 901)):
        s = 'summer'
    elif ((md >= 901) and (md <= 1130)):
        s = 'fall'
    elif ((md > 1130) and (md <= 229)):
        s = 'winter'
    else:
        raise IndexError("Invalid date")

    return s


In [5]:
def check_null(df):
    nulls = df.isnull().sum().sum()
    print("Total NaN:",nulls)

def check_shape(df):
    rows = df.shape[0]
    columns = df.shape[1]
    print("Rows:",rows)
    print("Columns:", columns)
    
def describe(df):
    description= df.describe()
    print(description)

def remove_duplicates(df):
    before_d = len(df)
    df.drop_duplicates(inplace=True)
    after_d = len(df)
    print(str(before_d-after_d) + " duplicates were removed!")

def reset_index(df):
    df = df.reset_index(inplace=True)

def eda(df, drop_duplicates = True):
    check_null(df)
    check_shape(df)
    describe(df)
    
    if drop_duplicates == True:
        remove_duplicates(df)
        
    reset_index(df)

## 4. Data Imports and Overview<a id='Data Overview'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

Two sets of features will be generated, using different feature engineering strategies. This is to diversify our models (models that are not correlated perform better when stacking).

In [6]:
df_train_1 = pd.read_csv('./datasets/train.csv')
df_train_2 = pd.read_csv('./datasets/train.csv')
df_kag_1 = pd.read_csv('./datasets/test.csv')
df_kag_2 = pd.read_csv('./datasets/test.csv')
df_spray_1 = pd.read_csv('./datasets/spray.csv')
df_spray_2 = pd.read_csv('./datasets/spray.csv')
df_weather_1 = pd.read_csv('./datasets/weather.csv')
df_weather_2 = pd.read_csv('./datasets/weather.csv')

In [7]:
[col for col in df_train_1.columns if col not in df_kag_1.columns]

['NumMosquitos', 'WnvPresent']

In [8]:
[col for col in df_kag_1.columns if col not in df_train_1.columns]

['Id']

> The NumMosquitos feature appears in the train set, but not the test set.

### 4.1 Evaluating Shape, Duplicates, Missingness and Data Types of Datasets<a id='Evaluating Shape'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [9]:
pd.DataFrame([[df_train_1.shape, df_kag_1.shape, df_spray_1.shape, df_weather_1.shape],
            [df_train_1.isnull().sum().sum(), df_kag_1.isnull().sum().sum(), df_spray_1.isnull().sum().sum(), (df_weather_1 == 'M').sum().sum()],
             [df_train_1.duplicated().sum(), df_kag_1.duplicated().sum(), df_spray_1.duplicated().sum(), df_weather_1.duplicated().sum()]],
            columns = ['Train Set', 'Kaggle Test Set', 'Spray Set', 'Weather Set'],
            index = ['Shape', 'Missingness', 'Duplicates'])

C:\Users\Daniel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Train Set,Kaggle Test Set,Spray Set,Weather Set
Shape,"(10506, 12)","(116293, 11)","(14835, 4)","(2944, 22)"
Missingness,0,0,584,7415
Duplicates,813,0,541,0


In [10]:
pd.DataFrame([df_train_1.dtypes, df_kag_1.dtypes, df_weather_1.dtypes, df_spray_1.dtypes]).fillna('-')

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Id,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Time
0,object,object,object,int64,object,object,object,float64,float64,int64,int64,int64,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,object,object,object,int64,object,object,object,float64,float64,int64,-,-,int64,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,object,-,-,-,-,-,-,-,-,-,-,-,-,int64,int64,int64,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,float64,int64,object,-
3,object,-,-,-,-,-,-,float64,float64,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,object


In [11]:
df_train_1['WnvPresent'].value_counts()

0    9955
1     551
Name: WnvPresent, dtype: int64

### 4.2 Investigating Duplicates <a id='Investigating Duplicates'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [12]:
df_train_1[df_train_1.duplicated()].head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
99,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,1,0
295,2007-07-11,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,50,0
351,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0
353,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0
512,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",CULEX RESTUANS,33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",41.883284,-87.705085,8,1,0


> Train duplicates are the result of the number of mosquitos overflowing into the next row (when greater than 50).

In [13]:
df_spray_1[df_spray_1.duplicated()].head()

,Date,Time,Latitude,Longitude
485,2011-09-07,7:43:40 PM,41.983917,-87.793088
490,2011-09-07,7:44:32 PM,41.986460,-87.794225
491,2011-09-07,7:44:32 PM,41.986460,-87.794225
492,2011-09-07,7:44:32 PM,41.986460,-87.794225
493,2011-09-07,7:44:32 PM,41.986460,-87.794225


> Spray duplicates may be dropped, as these are true duplicates.

In [14]:
remove_duplicates(df_spray_1)

541 duplicates were removed!


### 4.3 Analysis of Datasets <a id='Analysis of Datasets'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

<b> Shape </b>

The kaggle set is significantly larger than the train set given.

<b> Missingness </b>

Missing values - do they matter? Can they be imputed?

<b> Duplicates </b>

Duplicates for train set represent multiple collections of mosquitos. This is due to the mosquito collection process - the WNV test processes mosquito batches of 50 at a time, so traps with high numbers of mosquitos will have multiple rows. We will create two datasets with different strategies. The first approach is to sum up the rows to get an aggregate number of mosquitos, and the second is to just drop these duplicates.

Imbalanced classes - possible options:
<li>Change the algorithm
<li>Oversample minority class
<li>Undersample majority class
<li>Generate synthetic samples

## 5. Feature Engineering - Model 1 <a id='Model 1'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

### 5.1 Correcting Date Time Data

In [15]:
#Date is stored as a string. Convert to datetime object, make sure to remove time data since it's not relevant
df_train_1['Date'] = df_train_1['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_spray_1['Date'] = df_spray_1['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_weather_1['Date'] = df_weather_1['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_kag_1['Date'] = df_kag_1['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [16]:
#won't be used in modelling since it will correlate greatly with month - this is for EDA
df_train_1['Season'] = df_train_1['Date'].apply(lambda date: get_season(date))
df_kag_1['Season'] = df_kag_1['Date'].apply(lambda date: get_season(date))
df_weather_1['Season'] = df_weather_1['Date'].apply(lambda date: get_season(date))

In [17]:
df_train_1['Year'] = df_train_1['Date'].dt.year
df_spray_1['Year'] = df_spray_1['Date'].dt.year
df_kag_1['Year'] = df_kag_1['Date'].dt.year
df_weather_1['Year'] = df_weather_1['Date'].dt.year

In [18]:
df_train_1['Month'] = df_train_1['Date'].dt.month
df_kag_1['Month'] = df_kag_1['Date'].dt.month

In [19]:
df_spray_1.drop_duplicates(inplace = True)
df_spray_1.reset_index(inplace=True)

### 5.2 Grouping Mosquitos <a id='Grouping Mosquitos'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [20]:
#Combine alllllllll the mosquitos
df_train_1 = df_train_1.groupby(list(df_train_1.drop(['NumMosquitos', 'WnvPresent'], axis = 1).columns))['NumMosquitos','WnvPresent'].agg('sum').reset_index()

df_train_1['WnvPresent'] = df_train_1['WnvPresent'].apply(lambda x: 1 if x >= 1 else 0)

### 5.3 Evaluating Differences in Weather Stations <a id='Diff Weather'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [21]:
pd.DataFrame([df_weather_1[df_weather_1['Station']==1].dtypes,
             (df_weather_1[df_weather_1['Station']==1] == 'M').sum(),
             (df_weather_1[df_weather_1['Station']==1] == '  T').sum(),
             (df_weather_1[df_weather_1['Station']==1] == '-').sum()],
            index = ['DType', 'NA M', 'NA T', 'NA -'])

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Season,Year
DType,int64,datetime64[ns],int64,int64,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,float64,int64,object,object,int64
NA M,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1472,0,0,2,5,0,0,0,0,0
NA T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,163,0,0,0,0,0,0,0
NA -,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
pd.DataFrame([df_weather_1[df_weather_1['Station']==2].dtypes,
             (df_weather_1[df_weather_1['Station']==2] == 'M').sum(),
             (df_weather_1[df_weather_1['Station']==2] == '  T').sum(),
             (df_weather_1[df_weather_1['Station']==2] == '-').sum()],
            index = ['DType', 'NA M', 'NA T', 'NA -'])

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Season,Year
DType,int64,datetime64[ns],int64,int64,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,float64,int64,object,object,int64
NA M,0,0,0,0,11,1472,0,1,11,11,0,0,0,1472,1472,1472,2,2,4,0,0,3,0,0
NA T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,155,0,0,0,0,0,0,0
NA -,0,0,0,0,0,0,0,0,0,0,1472,1472,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df_weather_1[df_weather_1['Station']==1].describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir,Year
count,1472.0,1472.00000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000
mean,1.0,75.97894,56.730978,53.377717,6.987092,17.656250,2010.500000
std,0.0,11.50315,10.347907,10.670263,3.586353,10.222077,2.292067
min,1.0,42.00000,29.000000,22.000000,0.100000,1.000000,2007.000000
25%,1.0,68.00000,49.000000,46.000000,4.400000,7.000000,2008.750000
50%,1.0,78.00000,58.000000,54.000000,6.400000,19.500000,2010.500000
75%,1.0,85.00000,65.000000,61.250000,9.100000,26.000000,2012.250000
max,1.0,103.00000,82.000000,75.000000,24.100000,36.000000,2014.000000


In [24]:
df_weather_1[df_weather_1['Station']==2].describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir,Year
count,1472.0,1472.000000,1472.000000,1472.000000,1472.000000,1472.00000,1472.000000
mean,2.0,76.353261,58.889946,53.538043,6.934239,17.33356,2010.500000
std,0.0,11.421482,10.306855,10.683121,3.589725,9.90345,2.292067
min,2.0,41.000000,29.000000,23.000000,0.300000,1.00000,2007.000000
25%,2.0,69.000000,51.000000,46.000000,4.275000,8.00000,2008.750000
50%,2.0,78.000000,60.000000,55.000000,6.500000,19.00000,2010.500000
75%,2.0,85.000000,67.000000,62.000000,9.225000,24.00000,2012.250000
max,2.0,104.000000,83.000000,74.000000,21.700000,36.00000,2014.000000


> Station 2 has a lot more missing values, but we will be using both stations in our model as the differences in the weather data between the stations are non trivial. We will therefore use distance from station as a metric to determine which weather data use for each observation.

In [25]:
#Filling missing values with np.nan to get a clear picture of missingness
#T (trace) will be replaced with 0.01 as a dummy value, to represent trace amounts
df_weather_1 = df_weather_1.replace('M', np.nan).replace('-', np.nan).replace('  T', 0.01)

In [26]:
pd.DataFrame(df_weather_1.isnull().sum(), columns = ['Missing']).T

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Season,Year
Missing,0,0,0,0,11,1472,0,4,11,11,1472,1472,0,1472,2944,1472,2,4,9,0,0,3,0,0


In [27]:
df_weather_1.head(8)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Season,Year
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2,spring,2007
1,2,2007-05-01,84,52,68,NaN,51,57,0,3,NaN,NaN,,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6,spring,2007
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4,spring,2007
3,2,2007-05-02,60,43,52,NaN,42,47,13,0,NaN,NaN,BR HZ,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4,spring,2007
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9,spring,2007
5,2,2007-05-03,67,48,58,NaN,40,50,7,0,NaN,NaN,HZ,NaN,NaN,NaN,0.00,29.46,30.12,12.9,6,13.2,spring,2007
6,1,2007-05-04,66,49,58,4,41,50,7,0,0444,1852,RA,0,NaN,0.0,0.01,29.31,30.05,10.4,8,10.8,spring,2007
7,2,2007-05-04,78,51,NaN,NaN,42,50,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,0.00,29.36,30.04,10.1,7,10.4,spring,2007


In [28]:
df_weather_1['Tavg'].fillna((df_weather_1['Tmax'] + df_weather_1['Tmin'])/2, inplace = True)

In [29]:
#Drop depth, water and, and snowfall - these are winter/cold indicators, but the data is mostly from the warm periods.
df_weather_1 = df_weather_1.drop(['Depth', 'Water1', 'SnowFall'], axis = 1)

In [30]:
df_weather_1 = df_weather_1.reset_index()

In [31]:
for col in df_weather_1.columns:
    if df_weather_1[col].dtype == 'object':
        try: #Easier to catch the columns that cannot be converted to floats, e.g. the CodeSum col
            df_weather_1[col] = df_weather_1[col].astype(float)
        except:
            pass

In [32]:
#Relevant codes:
#TS - Thunderstorm
#RA - Raind
#DZ - Drizzle
#BR - Mist
#HZ - Haze


codes_to_dummy = ['TS', 'RA', 'DZ', 'BR', 'HZ']

for code in codes_to_dummy:
    df_weather_1[code] = df_weather_1['CodeSum'].str.contains(code, regex = False).astype(int)
    
df_weather_1 = df_weather_1.drop('CodeSum', axis = 1)

In [33]:
def f_to_celc(f):
    return np.round((5/9) * (f - 32), 2)

In [34]:
#Values are calculated using the August-Roche-Magnus approximation
def saturated_vapor_pressure(temp):
    temp_c = f_to_celc(temp)
    saturated_vapor_pressure = math.exp((17.625 * temp_c) / (243.04 + temp_c))
    return saturated_vapor_pressure

def actual_vapor_pressure(dew_point):
    dew_point_c = f_to_celc(dew_point)
    actual_vapor_pressure = math.exp((17.625 * dew_point_c) / (243.04 + dew_point_c))
    return actual_vapor_pressure

In [35]:
df_weather_1['humidity'] = np.round(df_weather_1['DewPoint'].apply(
    lambda y: actual_vapor_pressure(y)) / df_weather_1['Tavg'].apply(
    lambda x: saturated_vapor_pressure(x)), 3)

In [36]:
#Split into two dataframes for easier comparison, and to perform a rolling average for each station
df_weather_A = df_weather_1[df_weather_1['Station']==1]
df_weather_B = df_weather_1[df_weather_1['Station']==2]
df_weather_A = df_weather_A.set_index(df_weather_A['Date']).drop('index', axis = 1)
df_weather_B = df_weather_B.set_index(df_weather_B['Date']).drop('index', axis = 1)

In [37]:
df_weather_B['Depart'].fillna(df_weather_A['Depart'], inplace = True)
df_weather_B['Sunrise'].fillna(df_weather_A['Sunrise'], inplace = True)
df_weather_B['Sunset'].fillna(df_weather_A['Sunset'], inplace = True)

In [38]:
#fill with mean - maybe do an iterative imputation if im not lazy
df_weather_A['WetBulb'] = df_weather_A['WetBulb'].fillna(np.round(df_weather_A['WetBulb'].mean(), 1))
df_weather_A['PrecipTotal'] = df_weather_A['PrecipTotal'].fillna(np.round(df_weather_A['PrecipTotal'].mean(), 1))
df_weather_A['StnPressure'] = df_weather_A['StnPressure'].fillna(np.round(df_weather_A['StnPressure'].mean(), 1))

df_weather_B['WetBulb'] = df_weather_B['WetBulb'].fillna(np.round(df_weather_B['WetBulb'].mean(), 1))
df_weather_B['PrecipTotal'] = df_weather_B['PrecipTotal'].fillna(np.round(df_weather_B['PrecipTotal'].mean(), 1))
df_weather_B['StnPressure'] = df_weather_B['StnPressure'].fillna(np.round(df_weather_B['StnPressure'].mean(), 1))
df_weather_B['AvgSpeed'] = df_weather_B['AvgSpeed'].fillna(np.round(df_weather_B['AvgSpeed'].mean(), 1))

In [39]:
#Using string slice method because, for god knows what reason, 7 pm is stored as 1860 hours
def fix_nonsense_time(time):
    #There are only 3 problematic times (1660, 1760 and 1860). Rather than slicing and using up more computational time,
    #I will just hardcode it
    if time == 1860:
        time = datetime.datetime.strptime(str('1900'), "%H%M")

    elif time == 1760:
        time = datetime.datetime.strptime(str('1800'), "%H%M")

    elif time == 1660:
        time = datetime.datetime.strptime(str('1700'), "%H%M")        

    else:
        time = datetime.datetime.strptime(str(time), "%H%M")
        
    return time

In [40]:
df_weather_A['Sunset'] = df_weather_A['Sunset'].astype(int).apply(lambda x: fix_nonsense_time(x))
df_weather_B['Sunset'] = df_weather_B['Sunset'].astype(int).apply(lambda x: fix_nonsense_time(x))

In [41]:
df_weather_A['Sunrise'] = df_weather_A['Sunrise'].astype(int).apply(lambda x: fix_nonsense_time(x))
df_weather_B['Sunrise'] = df_weather_B['Sunrise'].astype(int).apply(lambda x: fix_nonsense_time(x))

In [42]:
df_weather_A['Temp_diff'] = df_weather_A['Tmax'] - df_weather_A['Tmin']
df_weather_B['Temp_diff'] = df_weather_B['Tmax'] - df_weather_B['Tmin']

In [43]:
df_weather_A['daylight_hours'] = df_weather_A['Sunset'] - df_weather_A['Sunrise']
df_weather_B['daylight_hours'] = df_weather_B['Sunset'] - df_weather_B['Sunrise']
df_weather_A['daylight_hours'] = df_weather_A['daylight_hours'].apply(lambda x: x.total_seconds()/3600)
df_weather_B['daylight_hours'] = df_weather_B['daylight_hours'].apply(lambda x: x.total_seconds()/3600)

In [44]:
df_weather_A = df_weather_A.drop(['Sunrise', 'Sunset'], axis = 1)
df_weather_B = df_weather_B.drop(['Sunrise', 'Sunset'], axis = 1)

In [45]:
#The entire life cycle, from an egg to an adult, takes approximately 8-10 days. We'll use 8.
df_weather_A['past_week_tavg'] = df_weather_A['Tavg'].rolling(window = 8).mean()
df_weather_A['past_week_precip'] = df_weather_A['PrecipTotal'].rolling(window = 8).sum()
df_weather_A['past_week_humid'] = df_weather_A['humidity'].rolling(window = 8).mean()
df_weather_B['past_week_tavg'] = df_weather_B['Tavg'].rolling(window = 8).mean()
df_weather_B['past_week_precip'] = df_weather_B['PrecipTotal'].rolling(window = 8).sum()
df_weather_B['past_week_humid'] = df_weather_B['humidity'].rolling(window = 8).mean()

In [46]:
df_weather_A['past_mth_tavg'] = df_weather_A['Tavg'].rolling(window = 30).mean()
df_weather_A['past_mth_precip'] = df_weather_A['PrecipTotal'].rolling(window = 30).sum()
df_weather_A['past_mth_humid'] = df_weather_A['humidity'].rolling(window = 30).mean()
df_weather_B['past_mth_tavg'] = df_weather_B['Tavg'].rolling(window = 30).mean()
df_weather_B['past_mth_precip'] = df_weather_B['PrecipTotal'].rolling(window = 30).sum()
df_weather_B['past_mth_humid'] = df_weather_B['humidity'].rolling(window = 30).mean()

In [47]:
df_weather_A[['TS_roll', 'RA_roll', 'DZ_roll']] = df_weather_A[['TS', 'RA', 'DZ']].rolling(window = 8).mean() > 0

df_weather_B[['TS_roll', 'RA_roll', 'DZ_roll']] = df_weather_B[['TS', 'RA', 'DZ']].rolling(window = 8).mean() > 0

In [48]:
df_weather_A['rained_last_week'] = df_weather_A[['TS_roll', 'RA_roll', 'DZ_roll']].any(axis = 1).astype(int)
df_weather_B['rained_last_week'] = df_weather_B[['TS_roll', 'RA_roll', 'DZ_roll']].any(axis = 1).astype(int)

In [49]:
df_weather_A['past_3_tavg'] = df_weather_A['Tavg'].rolling(window = 3).mean()
df_weather_A['past_3_precip'] = df_weather_A['PrecipTotal'].rolling(window = 3).sum()
df_weather_A['past_3_humid'] = df_weather_A['humidity'].rolling(window = 3).mean()
df_weather_A['past_3_wind'] = df_weather_A['AvgSpeed'].rolling(window = 3).mean()
df_weather_B['past_3_tavg'] = df_weather_B['Tavg'].rolling(window = 3).mean()
df_weather_B['past_3_precip'] = df_weather_B['PrecipTotal'].rolling(window = 3).sum()
df_weather_B['past_3_humid'] = df_weather_B['humidity'].rolling(window = 3).mean()
df_weather_B['past_3_wind'] = df_weather_B['AvgSpeed'].rolling(window = 3).mean()

In [50]:
df_weather_A = df_weather_A.drop('Date', axis = 1).reset_index()
df_weather_A.head()

,Date,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Season,Year,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,TS_roll,RA_roll,DZ_roll,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind
0,2007-05-01,1,83,50,67.0,14.0,51,56.0,0.0,2.0,0.00,29.10,29.82,1.7,27,9.2,spring,2007,0,0,0,0,0,0.565,33,14.016667,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,NaN,NaN,NaN,NaN
1,2007-05-02,1,59,42,51.0,-3.0,42,47.0,14.0,0.0,0.00,29.38,30.09,13.0,4,13.4,spring,2007,0,0,0,1,0,0.712,17,14.050000,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,NaN,NaN,NaN,NaN
2,2007-05-03,1,66,46,56.0,2.0,40,48.0,9.0,0.0,0.00,29.39,30.12,11.7,7,11.9,spring,2007,0,0,0,0,0,0.549,20,14.083333,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,58.0,0.00,0.608667,11.500000
3,2007-05-04,1,66,49,58.0,4.0,41,50.0,7.0,0.0,0.01,29.31,30.05,10.4,8,10.8,spring,2007,0,1,0,0,0,0.531,17,14.133333,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,55.0,0.01,0.597333,12.033333
4,2007-05-05,1,66,53,60.0,5.0,38,49.0,5.0,0.0,0.01,29.40,30.10,11.7,7,12.0,spring,2007,0,0,0,0,0,0.439,13,14.166667,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,58.0,0.02,0.506333,11.566667


In [51]:
df_weather_B = df_weather_B.drop('Date', axis = 1).reset_index()
df_weather_B.head()

,Date,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Season,Year,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,TS_roll,RA_roll,DZ_roll,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind
0,2007-05-01,2,84,52,68.0,14.0,51,57.0,0.0,3.0,0.00,29.18,29.82,2.7,25,9.6,spring,2007,0,0,0,0,0,0.545,32,14.016667,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,NaN,NaN,NaN,NaN
1,2007-05-02,2,60,43,52.0,-3.0,42,47.0,13.0,0.0,0.00,29.44,30.08,13.3,2,13.4,spring,2007,0,0,0,1,1,0.686,17,14.050000,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,NaN,NaN,NaN,NaN
2,2007-05-03,2,67,48,58.0,2.0,40,50.0,7.0,0.0,0.00,29.46,30.12,12.9,6,13.2,spring,2007,0,0,0,0,1,0.511,19,14.083333,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,59.333333,0.00,0.580667,12.066667
3,2007-05-04,2,78,51,64.5,4.0,42,50.0,NaN,NaN,0.00,29.36,30.04,10.1,7,10.4,spring,2007,0,0,0,0,0,0.438,27,14.133333,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,58.166667,0.00,0.545000,12.333333
4,2007-05-05,2,66,54,60.0,5.0,39,50.0,5.0,0.0,0.01,29.46,30.09,11.2,7,11.5,spring,2007,0,0,0,0,0,0.457,12,14.166667,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,0,60.833333,0.01,0.468667,11.700000


In [52]:
df_weather_1 = df_weather_A.append(df_weather_B).reset_index()

In [53]:
df_weather_1.columns

Index(['index', 'Date', 'Station', 'Tmax', 'Tmin', 'Tavg', 'Depart',
       'DewPoint', 'WetBulb', 'Heat', 'Cool', 'PrecipTotal', 'StnPressure',
       'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'Season', 'Year',
       'TS', 'RA', 'DZ', 'BR', 'HZ', 'humidity', 'Temp_diff', 'daylight_hours',
       'past_week_tavg', 'past_week_precip', 'past_week_humid',
       'past_mth_tavg', 'past_mth_precip', 'past_mth_humid', 'TS_roll',
       'RA_roll', 'DZ_roll', 'rained_last_week', 'past_3_tavg',
       'past_3_precip', 'past_3_humid', 'past_3_wind'],
      dtype='object')

In [54]:
df_weather_1.drop(['index', 'Tmax', 'Tmin', 'DewPoint', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
                'TS_roll', 'RA_roll', 'DZ_roll', 'Season'], axis = 1, inplace = True)

In [55]:
df_weather_1.head()

,Date,Station,Tavg,Depart,WetBulb,Heat,Cool,PrecipTotal,AvgSpeed,Year,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind
0,2007-05-01,1,67.0,14.0,56.0,0.0,2.0,0.00,9.2,2007,0,0,0,0,0,0.565,33,14.016667,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,2007-05-02,1,51.0,-3.0,47.0,14.0,0.0,0.00,13.4,2007,0,0,0,1,0,0.712,17,14.050000,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2,2007-05-03,1,56.0,2.0,48.0,9.0,0.0,0.00,11.9,2007,0,0,0,0,0,0.549,20,14.083333,NaN,NaN,NaN,NaN,NaN,NaN,0,58.0,0.00,0.608667,11.500000
3,2007-05-04,1,58.0,4.0,50.0,7.0,0.0,0.01,10.8,2007,0,1,0,0,0,0.531,17,14.133333,NaN,NaN,NaN,NaN,NaN,NaN,0,55.0,0.01,0.597333,12.033333
4,2007-05-05,1,60.0,5.0,49.0,5.0,0.0,0.01,12.0,2007,0,0,0,0,0,0.439,13,14.166667,NaN,NaN,NaN,NaN,NaN,NaN,0,58.0,0.02,0.506333,11.566667


In [56]:
df_yearly_avg = pd.DataFrame(df_weather_1.groupby('Year')[['Tavg', 'PrecipTotal']].agg(
    'mean')).sort_values(by = 'Year').reset_index()
df_yearly_avg

,Year,Tavg,PrecipTotal
0,2007,69.177989,0.110245
1,2008,66.024457,0.155516
2,2009,64.364130,0.130897
3,2010,68.687500,0.154158
4,2011,67.023098,0.144783
5,2012,69.092391,0.091603
6,2013,66.974185,0.109973
7,2014,66.570652,0.161929


In [57]:
df_weather_1.isnull().sum()

Date                 0
Station              0
Tavg                 0
Depart               0
WetBulb              0
Heat                11
Cool                11
PrecipTotal          0
AvgSpeed             0
Year                 0
TS                   0
RA                   0
DZ                   0
BR                   0
HZ                   0
humidity             0
Temp_diff            0
daylight_hours       0
past_week_tavg      14
past_week_precip    14
past_week_humid     14
past_mth_tavg       58
past_mth_precip     58
past_mth_humid      58
rained_last_week     0
past_3_tavg          4
past_3_precip        4
past_3_humid         4
past_3_wind          4
dtype: int64

Duplicates will only be dropped after merging with the train set, since the dates associated with the missing data may not be in the train set.

### 5.4 Cleaning Train Data <a id='Clean Train'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [58]:
df_train_1= df_train_1.merge(df_yearly_avg, on = 'Year', how = 'left')
df_kag_1 = df_kag_1.merge(df_yearly_avg, on = 'Year', how = 'left')

In [59]:
#Assume 30 days of effectiveness
#Creates a list of dates (datetimeindex) for all effective days of spray spray

effectiveness_period = 30
dateindex_list = [pd.date_range(date, periods = effectiveness_period, freq = 'D') for date in set(df_spray_1['Date'])]

#clunky way to combine datetime indices
for n, item in enumerate(dateindex_list):
    if n == 0:
        dateindex = item
    elif n < len(dateindex_list)-1:
        dateindex = pd.DatetimeIndex.union(self = dateindex, other = dateindex_list[n+1])
    else:
        pass
        
df_spray_1['date_range'] = df_spray_1['Date'].apply(lambda x: pd.date_range(x, periods = effectiveness_period, freq = 'D'))   
dateindex

DatetimeIndex(['2011-08-29', '2011-08-30', '2011-08-31', '2011-09-01',
               '2011-09-02', '2011-09-03', '2011-09-04', '2011-09-05',
               '2011-09-06', '2011-09-07',
               ...
               '2013-09-25', '2013-09-26', '2013-09-27', '2013-09-28',
               '2013-09-29', '2013-09-30', '2013-10-01', '2013-10-02',
               '2013-10-03', '2013-10-04'],
              dtype='datetime64[ns]', length=111, freq=None)

In [60]:
#this thing is really clunky, takes awhile to run - cut down on loops?
#Set threshold to whatever distance (in km) you think the spray is effective

threshold = 0.3
sprayed_list = []
for row, date in enumerate(df_train_1['Date']):
    #Cut down on samples by filtering out all dates not in dateindex - fewer iterations
    #Will have to check date again by matching against the specific date range, but this makes it slightly faster
    if date in dateindex:
        house_loc = (df_train_1['Latitude'][row], df_train_1['Longitude'][row])
        min_dist = 999999 #errr, placeholder number
        for spray_row, (lat_spray, long_spray) in enumerate(zip(df_spray_1['Latitude'], df_spray_1['Longitude'])):
            spray_loc = (lat_spray, long_spray)
            haversine_dist = haversine(house_loc, spray_loc)
            if haversine_dist < min_dist and date in df_spray_1['date_range'][spray_row]:
                min_dist = haversine_dist


        if threshold <= min_dist:
            sprayed_list.append(1)

        else:
            #out of range (haversine dist > threshold)
            sprayed_list.append(0)
            
    else:
        #out of range (date not during any of the spray dates)
        sprayed_list.append(0)
            


sprayed_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [61]:
df_train_1 = df_train_1.assign(is_sprayed = sprayed_list)
df_kag_1 = df_kag_1.assign(is_sprayed = 0)

In [62]:
#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

nearest_station = []
for row, date in enumerate(df_train_1['Date']):
    house_loc = (df_train_1['Latitude'][row], df_train_1['Longitude'][row])
    station_1_loc = (41.995, -87.933)
    station_2_loc = (41.786, -87.752)
    dist_1 = haversine(house_loc, station_1_loc)
    dist_2 = haversine(house_loc, station_2_loc)
    
    #if distance to station 1 > distance to station 2, append station 2 since it's nearer
    if dist_1 > dist_2:
        nearest_station.append(2)
        
    else:
        nearest_station.append(1)
    
df_train_1['Station'] = nearest_station

In [63]:
nearest_station = []
for row, date in enumerate(df_kag_1['Date']):
    house_loc = (df_kag_1['Latitude'][row], df_kag_1['Longitude'][row])
    station_1_loc = (41.995, -87.933)
    station_2_loc = (41.786, -87.752)
    dist_1 = haversine(house_loc, station_1_loc)
    dist_2 = haversine(house_loc, station_2_loc)
    
    if dist_1 > dist_2:
        nearest_station.append(2)
        
    else:
        nearest_station.append(1)
    
df_kag_1['Station'] = nearest_station

In [64]:
df_train_1 = df_train_1.merge(df_weather_1, on = ['Date', 'Station'])

In [65]:
df_kag_1 = df_kag_1.merge(df_weather_1, on = ['Date', 'Station'])

In [66]:
df_train_1.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Season,Year_x,Month,NumMosquitos,WnvPresent,Tavg_x,PrecipTotal_x,is_sprayed,Station,Tavg_y,Depart,WetBulb,Heat,Cool,PrecipTotal_y,AvgSpeed,Year_y,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,2007,0,0,0,1,1,0.538,23,14.933333,70.0,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,spring,2007,5,2,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,2007,0,0,0,1,1,0.538,23,14.933333,70.0,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667
2,2007-05-29,"1100 South Peoria Street, Chicago, IL 60608, USA",CULEX RESTUANS,11,S PEORIA ST,T091,"1100 S PEORIA ST, Chicago, IL",41.862292,-87.648860,8,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,2007,0,0,0,1,1,0.538,23,14.933333,70.0,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667
3,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,11,W CHICAGO,T049,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,2007,0,0,0,1,1,0.538,23,14.933333,70.0,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667
4,2007-05-29,"1500 North Long Avenue, Chicago, IL 60651, USA",CULEX RESTUANS,15,N LONG AVE,T153,"1500 N LONG AVE, Chicago, IL",41.907645,-87.760886,8,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,2007,0,0,0,1,1,0.538,23,14.933333,70.0,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667


In [67]:
df_kag_1.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Season,Year_x,Month,Tavg_x,PrecipTotal_x,is_sprayed,Station,Tavg_y,Depart,WetBulb,Heat,Cool,PrecipTotal_y,AvgSpeed,Year_y,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,summer,2008,6,66.024457,0.155516,0,1,74.0,7.0,64.0,0.0,9.0,0.0,10.0,2008,0,0,0,0,0,0.534,25,15.166667,73.125,2.16,0.7315,61.566667,3.82,0.639733,1,71.0,0.15,0.659667,9.1
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,summer,2008,6,66.024457,0.155516,0,1,74.0,7.0,64.0,0.0,9.0,0.0,10.0,2008,0,0,0,0,0,0.534,25,15.166667,73.125,2.16,0.7315,61.566667,3.82,0.639733,1,71.0,0.15,0.659667,9.1
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,summer,2008,6,66.024457,0.155516,0,1,74.0,7.0,64.0,0.0,9.0,0.0,10.0,2008,0,0,0,0,0,0.534,25,15.166667,73.125,2.16,0.7315,61.566667,3.82,0.639733,1,71.0,0.15,0.659667,9.1
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,summer,2008,6,66.024457,0.155516,0,1,74.0,7.0,64.0,0.0,9.0,0.0,10.0,2008,0,0,0,0,0,0.534,25,15.166667,73.125,2.16,0.7315,61.566667,3.82,0.639733,1,71.0,0.15,0.659667,9.1
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,summer,2008,6,66.024457,0.155516,0,1,74.0,7.0,64.0,0.0,9.0,0.0,10.0,2008,0,0,0,0,0,0.534,25,15.166667,73.125,2.16,0.7315,61.566667,3.82,0.639733,1,71.0,0.15,0.659667,9.1


In [68]:
df_train_1.drop(['Year_y','Address', 'Block', 'Street',
                 'AddressNumberAndStreet', 'AddressAccuracy'], axis = 1, inplace = True)

In [69]:
df_kag_1.drop(['Year_y', 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], axis = 1,
             inplace = True)

In [70]:
df_train_1.rename({'Year_x':'Year'}, axis = 1, inplace = True)
df_kag_1.rename({'Year_x':'Year'}, axis = 1, inplace = True)

### 5.5 Evaluating Geography <a id='Eval Geog'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

#### 5.5.1 Evaluating WNV by distance from high risk areas

The high counts of WNV may be assumed to be a function of both number of mosquitos and the presence of WNV infected birds. Here, we take a look at what traps appear in the confluence of mosquito and bird.

We will take only the traps that appear in all four years in the train set (2007, 2009, 2011, and 2013), so as to avoid overfitting to a certain year. This is made more complicated by how certain traps only appear in certain years (e.g. T900, the airport, only appears after 2007). We will run a few variations of this analysis to get a clearer picture of the distribution.

The goal of doing this is to use these traps as epicentres, and to evaluate WNV risk by distance from these epicentres.

In [71]:
wnv_2007 = df_train_1[df_train_1['Year']==2007].groupby([
    'Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(
    by = 'WnvPresent', ascending = False)

wnv_2007 = set(wnv_2007[wnv_2007['WnvPresent'] > 0].index)

In [72]:
wnv_2009 = df_train_1[df_train_1['Year']==2009].groupby([
    'Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(
    by = 'WnvPresent', ascending = False)

wnv_2009 = set(wnv_2009[wnv_2009['WnvPresent'] > 0].index)

In [73]:
wnv_2011 = df_train_1[df_train_1['Year']==2011].groupby([
    'Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(
    by = 'WnvPresent', ascending = False)

wnv_2011 = set(wnv_2011[wnv_2011['WnvPresent'] > 0].index)

In [74]:
wnv_2013 = df_train_1[df_train_1['Year']==2013].groupby([
    'Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(
    by = 'WnvPresent', ascending = False)

wnv_2013 = set(wnv_2013[wnv_2013['WnvPresent'] > 0].index)

In [75]:
set.intersection(wnv_2007, wnv_2009, wnv_2011, wnv_2013)

{('T002', 41.95469, -87.800991),
 ('T090', 41.868077, -87.666901),
 ('T095', 41.704336, -87.70373599999999),
 ('T158', 41.682587, -87.707973)}

#### 5.5.2 Evaluating NumMos by distance from high risk areas 

Here, we look at which traps capture the most number of mosquitos across the four years.

In [76]:
set2007 = df_train_1[df_train_1['Year']==2007].groupby(['Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(by = 'NumMosquitos', ascending = False).head(30)
traps_2007 = set(set2007.reset_index()['Trap'])
set2007

,,,WnvPresent,NumMosquitos
Trap,Latitude,Longitude,,
T115,41.673408,-87.599862,12,19134
T138,41.726465,-87.585413,9,9321
T103,41.702724,-87.536497,4,2988
T128,41.704572,-87.565666,5,2532
T212,41.680946,-87.535198,3,1785
T086,41.688324,-87.676709,7,1312
T135,41.662014,-87.724608,7,1235
T200,41.678618,-87.559308,1,899
T158,41.682587,-87.707973,1,892


In [77]:
set2009 = df_train_1[df_train_1['Year']==2009].groupby(['Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(by = 'NumMosquitos', ascending = False).head(16)
traps_2009 = set(set2009.reset_index()['Trap'])
set2009

,,,WnvPresent,NumMosquitos
Trap,Latitude,Longitude,,
T900,41.974689,-87.890615,2,5488
T135,41.662014,-87.724608,0,1178
T903,41.957799,-87.930995,2,1079
T225,41.743402,-87.731435,1,1060
T002,41.954690,-87.800991,1,818
T048,41.867108,-87.654224,0,739
T031,41.801498,-87.763416,1,621
T115,41.673408,-87.599862,0,529
T158,41.682587,-87.707973,1,483


In [78]:
set2011 = df_train_1[df_train_1['Year']==2011].groupby(['Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(by = 'NumMosquitos', ascending = False).head(22)
traps_2011 = set(set2011.reset_index()['Trap'])
set2011

,,,WnvPresent,NumMosquitos
Trap,Latitude,Longitude,,
T900,41.974689,-87.890615,12,4209
T114,41.798697,-87.736812,2,1313
T903,41.957799,-87.930995,7,1054
T115,41.673408,-87.599862,1,871
T151,41.916265,-87.800515,2,845
T223,41.973845,-87.805059,5,797
T003,41.964242,-87.757639,3,572
T008,42.008314,-87.777921,1,533
T031,41.801498,-87.763416,1,488


In [79]:
set2013 = df_train_1[df_train_1['Year']==2013].groupby(['Trap', 'Latitude', 'Longitude'])[['WnvPresent', 'NumMosquitos']].agg('sum').sort_values(by = 'NumMosquitos', ascending = False).head(35)
traps_2013 = set(set2013.reset_index()['Trap'])
set2013

,,,WnvPresent,NumMosquitos
Trap,Latitude,Longitude,,
T900,41.974689,-87.890615,15,5689
T002,41.954690,-87.800991,6,1792
T030,41.899230,-87.716788,7,1355
T008,42.008314,-87.777921,4,1162
T158,41.682587,-87.707973,2,1140
T115,41.673408,-87.599862,2,1134
T009,41.992478,-87.862995,4,1085
T227,41.728495,-87.600963,4,1011
T221,41.659112,-87.538693,3,934


In [80]:
intersect = set.intersection(traps_2007, traps_2009, traps_2011, traps_2013)
union = set.union(traps_2007, traps_2009, traps_2011, traps_2013)
intersect

{'T002', 'T008', 'T090', 'T115'}

<b> Tool to visualize trap distributions </b>
Input trap number into the field, and the widget will return a table of WnvPresent and NumMosquitos over the years where applicable. It will also plot these distributions against the mean.

In [81]:
sns.set_style("darkgrid")
df_temp = pd.DataFrame(df_train_1.groupby(['Trap', 'Year'])[['WnvPresent','NumMosquitos']].agg('sum')).reset_index()
df_temp = pd.DataFrame(df_temp.groupby('Year')[['WnvPresent', 'NumMosquitos']].agg('mean'),
                       columns = ['WnvPresent','NumMosquitos']).reset_index()
def trap_checker(var):
    df_mos_sum = pd.DataFrame(df_train_1[df_train_1['Trap'] == str(var)].groupby(
        'Year')['WnvPresent', 'NumMosquitos'].agg('sum')).reset_index().merge(df_yearly_avg, on = 'Year')
    print(df_mos_sum)

    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))
    #Plot average yearly num mos (all traps)
    sns.lineplot(x = list(df_temp['Year']), y = df_temp['NumMosquitos'], ax = ax[0], label = 'mean')    
    
    #Plot trap yearly num mos
    sns.lineplot(x = list(df_mos_sum['Year']),
                 y = list(df_mos_sum['NumMosquitos']), ax = ax[0], label = 'Trap '+str(var))

    
    ax[0].set_title("Number of Mosquitos by Year for Trap {}".format(var))
    ax[0].set_xlabel('Year')
    ax[0].set_ylabel('NumMosquitos') 
    ax[0].legend()
    
    sns.lineplot(x = list(df_temp['Year']), y = df_temp['WnvPresent'], ax = ax[1], label = 'mean')    
    
    #Plot trap yearly WnvPresent
    sns.lineplot(x = list(df_mos_sum['Year']),
                 y = list(df_mos_sum['WnvPresent']), ax = ax[1], label = 'Trap '+str(var))
        
    ax[1].set_title("Count of WnvPresent by Year for Trap {}".format(var))
    ax[1].set_xlabel('Year')
    ax[1].set_ylabel('WnvPresent') 
    ax[1].legend()
    
def trap_wrap(var):
    trap_checker(var)
    plt.show()

> Change var to trap number to update table and graphs:

In [82]:
interact(trap_wrap, var = 'T900');

interactive(children=(Text(value='T900', description='var'), Output()), _dom_classes=('widget-interact',))

Now that we have a set of epicentres, we will calculate the distance of each trap from an epicentre.

In [83]:
distance_to_T900 = []
distance_to_T002 = []
distance_to_T008 = []
distance_to_T115 = []
distance_to_T090 = []
distance_to_T095 = []
distance_to_T158 = []

for row, date in enumerate(df_train_1['Date']):
    house_loc = (df_train_1['Latitude'][row], df_train_1['Longitude'][row])
    T900_loc = (41.974689, -87.89061) 
    T002_loc = (41.954690, -87.800991) 
    T008_loc = (42.008314, -87.777921)
    T115_loc = (41.673408, -87.599862)
    T090_loc = (41.868077, -87.666901)
    T095_loc = (41.704336, -87.70373599999999)
    T158_loc = (41.682587, -87.707973)
    
    
    dist_1 = haversine(house_loc, T900_loc)
    dist_2 = haversine(house_loc, T002_loc)
    dist_3 = haversine(house_loc, T008_loc)
    dist_4 = haversine(house_loc, T115_loc) 
    dist_5 = haversine(house_loc, T090_loc) 
    dist_6 = haversine(house_loc, T095_loc) 
    dist_7 = haversine(house_loc, T158_loc) 
    
    distance_to_T900.append(dist_1)
    distance_to_T002.append(dist_2)
    distance_to_T008.append(dist_3)
    distance_to_T115.append(dist_4)
    distance_to_T090.append(dist_5)
    distance_to_T095.append(dist_6)
    distance_to_T158.append(dist_7)
    
df_train_1['distance_to_T900'] = distance_to_T900
df_train_1['distance_to_T002'] = distance_to_T002
df_train_1['distance_to_T008'] = distance_to_T008
df_train_1['distance_to_T115'] = distance_to_T115
df_train_1['distance_to_T090'] = distance_to_T090
df_train_1['distance_to_T095'] = distance_to_T095
df_train_1['distance_to_T158'] = distance_to_T158

In [84]:
distance_to_T900 = []
distance_to_T002 = []
distance_to_T008 = []
distance_to_T115 = []
distance_to_T090 = []
distance_to_T095 = []
distance_to_T158 = []


for row, date in enumerate(df_kag_1['Date']):
    house_loc = (df_kag_1['Latitude'][row], df_kag_1['Longitude'][row])
    T900_loc = (41.974689, -87.89061) #T900
    T002_loc = (41.954690, -87.800991) #T002
    T008_loc = (42.008314, -87.777921)
    T115_loc = (41.673408, -87.599862)
    T090_loc = (41.868077, -87.666901)
    T095_loc = (41.704336, -87.70373599999999)
    T158_loc = (41.682587, -87.707973)
    
    dist_1 = haversine(house_loc, T900_loc)
    dist_2 = haversine(house_loc, T002_loc)
    dist_3 = haversine(house_loc, T008_loc)
    dist_4 = haversine(house_loc, T115_loc)
    dist_5 = haversine(house_loc, T090_loc) 
    dist_6 = haversine(house_loc, T095_loc) 
    dist_7 = haversine(house_loc, T158_loc) 
    
    distance_to_T900.append(dist_1)
    distance_to_T002.append(dist_2)
    distance_to_T008.append(dist_3)
    distance_to_T115.append(dist_4)
    distance_to_T090.append(dist_5)
    distance_to_T095.append(dist_6)
    distance_to_T158.append(dist_7)
    
df_kag_1['distance_to_T900'] = distance_to_T900
df_kag_1['distance_to_T002'] = distance_to_T002
df_kag_1['distance_to_T008'] = distance_to_T008
df_kag_1['distance_to_T115'] = distance_to_T115
df_kag_1['distance_to_T090'] = distance_to_T090
df_kag_1['distance_to_T095'] = distance_to_T095
df_kag_1['distance_to_T158'] = distance_to_T158

#### One more check for missing values

In [85]:
df_kag_1.isnull().sum()

Id                  0
Date                0
Species             0
Trap                0
Latitude            0
Longitude           0
Season              0
Year                0
Month               0
Tavg_x              0
PrecipTotal_x       0
is_sprayed          0
Station             0
Tavg_y              0
Depart              0
WetBulb             0
Heat                0
Cool                0
PrecipTotal_y       0
AvgSpeed            0
TS                  0
RA                  0
DZ                  0
BR                  0
HZ                  0
humidity            0
Temp_diff           0
daylight_hours      0
past_week_tavg      0
past_week_precip    0
past_week_humid     0
past_mth_tavg       0
past_mth_precip     0
past_mth_humid      0
rained_last_week    0
past_3_tavg         0
past_3_precip       0
past_3_humid        0
past_3_wind         0
distance_to_T900    0
distance_to_T002    0
distance_to_T008    0
distance_to_T115    0
distance_to_T090    0
distance_to_T095    0
distance_t

In [86]:
df_train_1.isnull().sum()

Date                 0
Species              0
Trap                 0
Latitude             0
Longitude            0
Season               0
Year                 0
Month                0
NumMosquitos         0
WnvPresent           0
Tavg_x               0
PrecipTotal_x        0
is_sprayed           0
Station              0
Tavg_y               0
Depart               0
WetBulb              0
Heat                 0
Cool                 0
PrecipTotal_y        0
AvgSpeed             0
TS                   0
RA                   0
DZ                   0
BR                   0
HZ                   0
humidity             0
Temp_diff            0
daylight_hours       0
past_week_tavg       0
past_week_precip     0
past_week_humid      0
past_mth_tavg       25
past_mth_precip     25
past_mth_humid      25
rained_last_week     0
past_3_tavg          0
past_3_precip        0
past_3_humid         0
past_3_wind          0
distance_to_T900     0
distance_to_T002     0
distance_to_T008     0
distance_to

In [87]:
df_train_1[df_train_1.isnull().any(axis = 1)]

,Date,Species,Trap,Latitude,Longitude,Season,Year,Month,NumMosquitos,WnvPresent,Tavg_x,PrecipTotal_x,is_sprayed,Station,Tavg_y,Depart,WetBulb,Heat,Cool,PrecipTotal_y,AvgSpeed,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind,distance_to_T900,distance_to_T002,distance_to_T008,distance_to_T115,distance_to_T090,distance_to_T095,distance_to_T158
0,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,22.926134,15.567304,18.740850,22.005259,1.055244,18.559147,20.996372
1,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,spring,2007,5,2,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,22.926134,15.567304,18.740850,22.005259,1.055244,18.559147,20.996372
2,2007-05-29,CULEX RESTUANS,T091,41.862292,-87.648860,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,23.585867,16.249573,19.432126,21.392474,1.626558,18.143716,20.574840
3,2007-05-29,CULEX RESTUANS,T049,41.896282,-87.655232,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,21.332840,13.696506,16.066296,25.204128,3.281668,21.718848,24.160799
4,2007-05-29,CULEX RESTUANS,T153,41.907645,-87.760886,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,13.065187,6.194495,11.282176,29.267899,8.937944,23.097898,25.406864
5,2007-05-29,CULEX RESTUANS,T045,41.921600,-87.666455,spring,2007,5,2,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,19.455140,11.720575,13.338113,28.144422,5.951609,24.355462,26.798958
6,2007-05-29,CULEX RESTUANS,T094,41.720848,-87.666014,spring,2007,5,3,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,33.805206,28.304534,33.281002,7.615282,16.371306,3.629774,5.498602
7,2007-05-29,CULEX PIPIENS/RESTUANS,T050,41.919343,-87.694259,spring,2007,5,1,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,17.365830,9.663806,12.071561,28.444343,6.133828,23.920619,26.350634
8,2007-05-29,CULEX PIPIENS/RESTUANS,T054,41.921965,-87.632085,spring,2007,5,2,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,22.169522,14.437016,15.413515,27.767095,6.649014,24.917194,27.350837
9,2007-05-29,CULEX RESTUANS,T054,41.921965,-87.632085,spring,2007,5,3,0,69.177989,0.110245,0,2,77.0,10.0,66.0,0.0,12.0,0.0,7.4,0,0,0,1,1,0.538,23,14.933333,70.000,0.46,0.517375,NaN,NaN,NaN,1,69.666667,0.08,0.541333,7.266667,22.169522,14.437016,15.413515,27.767095,6.649014,24.917194,27.350837


These don't have WNV virus, and removing them may help balancing our classes somewhat.

In [88]:
df_train_1.dropna(axis = 0, inplace = True)

In [89]:
df_train_1.isnull().sum().sum()

0

### 5.6 Dummying Variables  <a id='Dummy Var'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [90]:
#For now, we will just dummy species:
df_train_1 = pd.get_dummies(df_train_1, columns = ['Species'])
df_kag_1 = pd.get_dummies(df_kag_1, columns = ['Species'])

In [91]:
print([col for col in df_train_1.columns if col not in df_kag_1.columns])
print([col for col in df_kag_1.columns if col not in df_train_1.columns])

['NumMosquitos', 'WnvPresent']
['Id', 'Species_UNSPECIFIED CULEX']


In [92]:
#Create dummy column for unspecified culex
df_train_1['Species_UNSPECIFIED CULEX'] = 0

In [93]:
df_train_1 = df_train_1.reset_index().drop('index', axis=1)
df_kag_1 = df_kag_1.reset_index().drop('index', axis=1)

In [94]:
def ungroup_mosq(df):
    for row in range(df.shape[0]):
        if df.loc[row,'Species_CULEX PIPIENS/RESTUANS'] == 1:
            df.loc[row,'Species_CULEX PIPIENS'] = 1
            df.loc[row,'Species_CULEX RESTUANS'] = 1
    df = df.drop(columns = ['Species_CULEX PIPIENS/RESTUANS'],inplace=True)
    
ungroup_mosq(df_train_1)
ungroup_mosq(df_kag_1)

In [95]:
df_train_1.head()

,Date,Trap,Latitude,Longitude,Season,Year,Month,NumMosquitos,WnvPresent,Tavg_x,PrecipTotal_x,is_sprayed,Station,Tavg_y,Depart,WetBulb,Heat,Cool,PrecipTotal_y,AvgSpeed,TS,RA,DZ,BR,HZ,humidity,Temp_diff,daylight_hours,past_week_tavg,past_week_precip,past_week_humid,past_mth_tavg,past_mth_precip,past_mth_humid,rained_last_week,past_3_tavg,past_3_precip,past_3_humid,past_3_wind,distance_to_T900,distance_to_T002,distance_to_T008,distance_to_T115,distance_to_T090,distance_to_T095,distance_to_T158,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX
0,2007-06-05,T048,41.867108,-87.654224,summer,2007,6,2,0,69.177989,0.110245,0,2,57.0,-9.0,52.0,8.0,0.0,0.27,8.1,0,1,0,1,0,0.692,12,15.1,71.875,1.65,0.630625,66.8,2.92,0.540033,1,64.0,0.9,0.718,8.466667,22.926134,15.567304,18.740850,22.005259,1.055244,18.559147,20.996372,0,1,0,0,0,0,0
1,2007-06-05,T048,41.867108,-87.654224,summer,2007,6,1,0,69.177989,0.110245,0,2,57.0,-9.0,52.0,8.0,0.0,0.27,8.1,0,1,0,1,0,0.692,12,15.1,71.875,1.65,0.630625,66.8,2.92,0.540033,1,64.0,0.9,0.718,8.466667,22.926134,15.567304,18.740850,22.005259,1.055244,18.559147,20.996372,0,1,1,0,0,0,0
2,2007-06-05,T048,41.867108,-87.654224,summer,2007,6,1,0,69.177989,0.110245,0,2,57.0,-9.0,52.0,8.0,0.0,0.27,8.1,0,1,0,1,0,0.692,12,15.1,71.875,1.65,0.630625,66.8,2.92,0.540033,1,64.0,0.9,0.718,8.466667,22.926134,15.567304,18.740850,22.005259,1.055244,18.559147,20.996372,0,0,1,0,0,0,0
3,2007-06-05,T092,41.869216,-87.627561,summer,2007,6,1,0,69.177989,0.110245,0,2,57.0,-9.0,52.0,8.0,0.0,0.27,8.1,0,1,0,1,0,0.692,12,15.1,71.875,1.65,0.630625,66.8,2.92,0.540033,1,64.0,0.9,0.718,8.466667,24.722348,17.212888,19.846973,21.893725,3.259986,19.391178,21.797215,0,1,1,0,0,0,0
4,2007-06-05,T049,41.896282,-87.655232,summer,2007,6,2,0,69.177989,0.110245,0,2,57.0,-9.0,52.0,8.0,0.0,0.27,8.1,0,1,0,1,0,0.692,12,15.1,71.875,1.65,0.630625,66.8,2.92,0.540033,1,64.0,0.9,0.718,8.466667,21.332840,13.696506,16.066296,25.204128,3.281668,21.718848,24.160799,0,0,1,0,0,0,0


In [96]:
df_train_1.to_csv("clean_train_1.csv")
df_kag_1.to_csv("clean_test_1.csv")

## 6. Feature Engineering - Model 2  <a id='Model 2'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [97]:
eda(df_train_2, drop_duplicates = False)

Total NaN: 0
Rows: 10506
Columns: 12
              Block      Latitude     Longitude  AddressAccuracy  \
count  10506.000000  10506.000000  10506.000000     10506.000000   
mean      35.687797     41.841139    -87.699908         7.819532   
std       24.339468      0.112742      0.096514         1.452921   
min       10.000000     41.644612    -87.930995         3.000000   
25%       12.000000     41.732984    -87.760070         8.000000   
50%       33.000000     41.846283    -87.694991         8.000000   
75%       52.000000     41.954690    -87.627796         9.000000   
max       98.000000     42.017430    -87.531635         9.000000   

       NumMosquitos    WnvPresent  
count  10506.000000  10506.000000  
mean      12.853512      0.052446  
std       16.133816      0.222936  
min        1.000000      0.000000  
25%        2.000000      0.000000  
50%        5.000000      0.000000  
75%       17.000000      0.000000  
max       50.000000      1.000000  


In [98]:
eda(df_kag_2)

Total NaN: 0
Rows: 116293
Columns: 11
                  Id          Block       Latitude      Longitude  \
count  116293.000000  116293.000000  116293.000000  116293.000000   
mean    58147.000000      41.131100      41.849389     -87.693658   
std     33571.041765      24.864726       0.106593       0.080699   
min         1.000000      10.000000      41.644612     -87.930995   
25%     29074.000000      18.000000      41.753411     -87.750938   
50%     58147.000000      39.000000      41.862292     -87.694991   
75%     87220.000000      61.000000      41.951866     -87.648860   
max    116293.000000      98.000000      42.017430     -87.531635   

       AddressAccuracy  
count    116293.000000  
mean          7.954357  
std           1.252733  
min           3.000000  
25%           8.000000  
50%           8.000000  
75%           9.000000  
max           9.000000  
0 duplicates were removed!


In [99]:
eda(df_spray_2)

Total NaN: 584
Rows: 14835
Columns: 4
           Latitude     Longitude
count  14835.000000  14835.000000
mean      41.904828    -87.736690
std        0.104381      0.067292
min       41.713925    -88.096468
25%       41.785001    -87.794225
50%       41.940075    -87.727853
75%       41.980978    -87.694108
max       42.395983    -87.586727
541 duplicates were removed!


In [100]:
eda(df_weather_2)

Total NaN: 0
Rows: 2944
Columns: 22
           Station         Tmax         Tmin     DewPoint  ResultSpeed  \
count  2944.000000  2944.000000  2944.000000  2944.000000  2944.000000   
mean      1.500000    76.166101    57.810462    53.457880     6.960666   
std       0.500085    11.461970    10.381939    10.675181     3.587527   
min       1.000000    41.000000    29.000000    22.000000     0.100000   
25%       1.000000    69.000000    50.000000    46.000000     4.300000   
50%       1.500000    78.000000    59.000000    54.000000     6.400000   
75%       2.000000    85.000000    66.000000    62.000000     9.200000   
max       2.000000   104.000000    83.000000    75.000000    24.100000   

         ResultDir  
count  2944.000000  
mean     17.494905  
std      10.063609  
min       1.000000  
25%       7.000000  
50%      19.000000  
75%      25.000000  
max      36.000000  
0 duplicates were removed!


### 6.1 Cleaning Weather Data  <a id='clean weather 2'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [101]:
#assign weather data to traps based on distance from weather station to trap

def find_haversine(lat1, lon1, lat2, lon2):
    '''Calculates the distance between two lat-long points.'''
    #https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
    R = 6378 #earth radius in kilometers

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

#assign weather station to trap based on location
def assign_station(df):
    stat1lat = 41.995
    stat1long = -87.933
    stat2lat = 41.786
    stat2long = -87.752 

    df['Station'] = 1

    for i in range(df.shape[0]):
        if find_haversine(stat1lat, stat1long, df.loc[i, 'Latitude'], df.loc[i, 'Longitude']) > find_haversine(stat2lat, stat2long, df.loc[i, 'Latitude'], df.loc[i, 'Longitude']):
            df.loc[i,'Station'] = 2

In [102]:
assign_station(df_train_2)
assign_station(df_kag_2)

In [103]:
print(df_train_2.head())
print(df_kag_2.head())

   index        Date                                             Address  \
0      0  2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634, USA   
1      1  2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634, USA   
2      2  2007-05-29   6200 North Mandell Avenue, Chicago, IL 60646, USA   
3      3  2007-05-29     7900 West Foster Avenue, Chicago, IL 60656, USA   
4      4  2007-05-29     7900 West Foster Avenue, Chicago, IL 60656, USA   

                  Species  Block           Street  Trap  \
0  CULEX PIPIENS/RESTUANS     41   N OAK PARK AVE  T002   
1          CULEX RESTUANS     41   N OAK PARK AVE  T002   
2          CULEX RESTUANS     62    N MANDELL AVE  T007   
3  CULEX PIPIENS/RESTUANS     79     W FOSTER AVE  T015   
4          CULEX RESTUANS     79     W FOSTER AVE  T015   

              AddressNumberAndStreet   Latitude  Longitude  AddressAccuracy  \
0  4100  N OAK PARK AVE, Chicago, IL  41.954690 -87.800991                9   
1  4100  N OAK PARK AVE, Chica

In [104]:
#drop columns not being used
df_weather_2.drop(columns = ['SeaLevel','CodeSum', 'Sunrise', 'Sunset','Depart','Depth','Water1', 'SnowFall', 'Cool', 'Heat'], inplace = True)

weather_columns = df_weather_2.columns.tolist()

In [105]:
weather_columns

['index',
 'Station',
 'Date',
 'Tmax',
 'Tmin',
 'Tavg',
 'DewPoint',
 'WetBulb',
 'PrecipTotal',
 'StnPressure',
 'ResultSpeed',
 'ResultDir',
 'AvgSpeed']

In [106]:
df_weather_2.loc[(df_weather_2['Tavg'] == 'M')]['Tavg']

7       M
505     M
675     M
1637    M
2067    M
2211    M
2501    M
2511    M
2525    M
2579    M
2811    M
Name: Tavg, dtype: object

In [107]:
#replace M in Tavg with the average of Tmax and Tmin on that day
for row in range(df_weather_2.shape[0]):
    if df_weather_2.loc[row,'Tavg'] == 'M':
        df_weather_2.loc[row,'Tavg'] = (df_weather_2.loc[row, 'Tmax']+df_weather_2.loc[row, 'Tmin'])/2


In [108]:
df_weather_2.loc[(df_weather_2['Tavg'] == 'M')]

,index,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,ResultSpeed,ResultDir,AvgSpeed


In [109]:
weather_columns.remove('Tavg')

In [110]:
#for missing data in weather columns, use previous day's data
for col in weather_columns:
    for row in range(df_weather_2.shape[0]):
        if df_weather_2.loc[row, col] == 'M' or df_weather_2.loc[row, col] == "  T":  #PrecipTotal has T for unvailable data
            df_weather_2.loc[row, col] = df_weather_2.loc[row - 1, col]

In [111]:
print(df_weather_2.dtypes)

cols_changetypes = ['Tavg','WetBulb','PrecipTotal','StnPressure','AvgSpeed']

for cols in cols_changetypes:
        df_weather_2[cols] = pd.to_numeric(df_weather_2[cols])

index            int64
Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
DewPoint         int64
WetBulb         object
PrecipTotal     object
StnPressure     object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object


In [112]:
df_weather_2.dtypes

index            int64
Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg           float64
DewPoint         int64
WetBulb          int64
PrecipTotal    float64
StnPressure    float64
ResultSpeed    float64
ResultDir        int64
AvgSpeed       float64
dtype: object

In [113]:
df_weather_2.to_csv("clean_weather_2.csv")

### 6.2 Cleaning up Train and Test Datasets  <a id='clean train 2'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [114]:
def get_dummies(df):   
    dummies=pd.get_dummies(df['Species'])
    df = pd.concat([df, dummies], axis=1)
    return df  

def trap_bias(df):
    form = lambda x: np.sum(x)/float(x.count())*100
    transformed = df[['Trap','WnvPresent']].groupby('Trap').agg(form)
    return transformed.reset_index().rename(columns={'WnvPresent':'Bias'})

def row_leakage(df):
    grouped = df[['Date', 'Trap', 'NumMosquitos','Species','WnvPresent','Address', 'Station']].groupby(by = ['Date','Trap','Address','Species', 'Station'])['NumMosquitos','WnvPresent'] \
    .sum().reset_index()
#     grouped = grouped.reset_index().rename(columns={'Address':'MCount'})
    return grouped

def MergeBias(df1, df2):
    df = df1.merge(df2, on='Trap', how="left",  left_index=True)
    return df

# def MergeRL(df1, df2):
#     df = df1.merge(df2, on=['Date','Trap'], how="left",  left_index=True)
#     return df

def drop_cols(df):
    col_filt = ['Address','Street','AddressNumberAndStreet','AddressAccuracy']
    df.drop(col_filt, axis=1, inplace=True)
    return df

def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['year'] = df.Date.dt.year
    df['month'] = df.Date.dt.month
    df['day'] = df.Date.dt.day
    df['week'] = df.Date.dt.week
    df.Date = df.Date.dt.date
    return df

def reset_index(df):
    df = df.reset_index()
    
    return df

def clean_data(train,test):
    train = row_leakage(train)
    train = get_dummies(train)
    bias = trap_bias(train)
    train = MergeBias(train,bias)
#     train = MergeRL(train,rl)
#     train = drop_cols(train)
    train = split_date(train)
    train = reset_index(train)
    
#    rl = row_leakage(test)
    test = get_dummies(test)
    test = MergeBias(test,bias)
#     test = MergeRL(test,rl)
#   test = drop_cols(test)
    test = split_date(test) 
    test = reset_index(test)
    return train,test
    

In [115]:
pd.set_option('display.max_columns',500)
df_train_2.head()

,index,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Station
0,0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,1
1,1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,1
2,2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,1
3,3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,1
4,4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,1


In [116]:
#Looking at mosquitos and wnv present by date by trap
df_train_2.groupby(by = ['Date', 'Trap'])['NumMosquitos', 'WnvPresent'].sum().sort_values('WnvPresent', ascending = False).head()

,,NumMosquitos,WnvPresent
Date,Trap,,
2013-08-29,T900,608,12
2007-08-16,T115,2636,11
2013-09-06,T900,550,9
2013-08-01,T900,984,9
2007-08-22,T138,1124,7


In [117]:
df_train_2.columns

Index(['index', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Station'],
      dtype='object')

In [118]:
#grouped = train.groupby(['Date','Trap','Species'])[['NumMosquitos']].sum()
#grouped
df_train_2[['Date', 'Trap', 'NumMosquitos','Species','WnvPresent','Address', 'Station']].groupby(by = ['Date','Trap','Address','Species', 'Station'])['NumMosquitos','WnvPresent'] \
    .sum().reset_index()#.sort_values('NumMosquitos', ascending = False)  #.set_index('Date')

,Date,Trap,Address,Species,Station,NumMosquitos,WnvPresent
0,2007-05-29,T002,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS/RESTUANS,1,1,0
1,2007-05-29,T002,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX RESTUANS,1,1,0
2,2007-05-29,T007,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,1,1,0
3,2007-05-29,T015,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,1,1,0
4,2007-05-29,T015,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,1,4,0
5,2007-05-29,T045,"1500 West Webster Avenue, Chicago, IL 60614, USA",CULEX RESTUANS,2,2,0
6,2007-05-29,T046,"2500 West Grand Avenue, Chicago, IL 60654, USA",CULEX RESTUANS,2,1,0
7,2007-05-29,T048,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,2,1,0
8,2007-05-29,T048,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,2,2,0
9,2007-05-29,T049,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,2,1,0


In [119]:
clean_train, clean_test = clean_data(df_train_2,df_kag_2)

In [120]:
def ungroup_mosq(df):
    for row in range(df.shape[0]):
        if df.loc[row,'CULEX PIPIENS/RESTUANS'] == 1:
            df.loc[row,'CULEX PIPIENS'] = 1
            df.loc[row,'CULEX RESTUANS'] = 1
    df = df.drop(columns = ['CULEX PIPIENS/RESTUANS','Species'],inplace=True)
    
ungroup_mosq(clean_train)
ungroup_mosq(clean_test)

In [121]:
clean_test.columns

Index(['level_0', 'index', 'Id', 'Date', 'Address', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'Station', 'CULEX ERRATICUS', 'CULEX PIPIENS', 'CULEX RESTUANS',
       'CULEX SALINARIUS', 'CULEX TARSALIS', 'CULEX TERRITANS',
       'UNSPECIFIED CULEX', 'Bias', 'year', 'month', 'day', 'week'],
      dtype='object')

In [122]:
clean_train.head()

,index,Date,Trap,Address,Station,NumMosquitos,WnvPresent,CULEX ERRATICUS,CULEX PIPIENS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,Bias,year,month,day,week
0,1,2007-05-29,T002,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",1,1,0,0,1,1,0,0,0,12.244898,2007,5,29,22
1,1,2007-05-29,T002,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",1,1,0,0,0,1,0,0,0,12.244898,2007,5,29,22
2,6,2007-05-29,T007,"6200 North Mandell Avenue, Chicago, IL 60646, USA",1,1,0,0,0,1,0,0,0,0.000000,2007,5,29,22
3,13,2007-05-29,T015,"7900 West Foster Avenue, Chicago, IL 60656, USA",1,1,0,0,1,1,0,0,0,12.500000,2007,5,29,22
4,13,2007-05-29,T015,"7900 West Foster Avenue, Chicago, IL 60656, USA",1,4,0,0,0,1,0,0,0,12.500000,2007,5,29,22


In [123]:
clean_test.head()

,level_0,index,Id,Date,Address,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Station,CULEX ERRATICUS,CULEX PIPIENS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,UNSPECIFIED CULEX,Bias,year,month,day,week
0,1,0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,1,1,0,0,0,0,12.244898,2008,6,11,24
1,1,1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,1,0,0,0,0,12.244898,2008,6,11,24
2,1,2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,1,0,0,0,0,0,12.244898,2008,6,11,24
3,1,3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,0,1,0,0,0,12.244898,2008,6,11,24
4,1,4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,0,0,0,1,0,12.244898,2008,6,11,24


In [124]:
clean_train.drop(['index'],axis=1,inplace=True)
clean_test.drop(['index','level_0'],axis=1,inplace=True)

In [125]:
for row in range(clean_train.shape[0]):
    if clean_train.loc[row,'WnvPresent'] >=1:
        clean_train.loc[row,'WnvPresent']=1

In [126]:
clean_train=pd.get_dummies(clean_train, columns = ['Trap'])
clean_test=pd.get_dummies(clean_test, columns = ['Trap'])

In [127]:
clean_train.groupby('WnvPresent').sum()

,Station,NumMosquitos,CULEX ERRATICUS,CULEX PIPIENS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,Bias,year,month,day,week,Trap_T001,Trap_T002,Trap_T003,Trap_T004,Trap_T005,Trap_T006,Trap_T007,Trap_T008,Trap_T009,Trap_T011,Trap_T012,Trap_T013,Trap_T014,Trap_T015,Trap_T016,Trap_T017,Trap_T018,Trap_T019,Trap_T025,Trap_T027,Trap_T028,Trap_T030,Trap_T031,Trap_T033,Trap_T034,Trap_T035,Trap_T036,Trap_T037,Trap_T039,Trap_T040,Trap_T043,Trap_T044,Trap_T045,Trap_T046,Trap_T047,Trap_T048,Trap_T049,Trap_T050,Trap_T051,Trap_T054,Trap_T054C,Trap_T060,Trap_T061,Trap_T062,Trap_T063,Trap_T065,Trap_T066,Trap_T067,Trap_T069,Trap_T070,Trap_T071,Trap_T072,Trap_T073,Trap_T074,Trap_T075,Trap_T076,Trap_T077,Trap_T078,Trap_T079,Trap_T080,Trap_T081,Trap_T082,Trap_T083,Trap_T084,Trap_T085,Trap_T086,Trap_T088,Trap_T089,Trap_T090,Trap_T091,Trap_T092,Trap_T094,Trap_T094B,Trap_T095,Trap_T096,Trap_T097,Trap_T099,Trap_T100,Trap_T102,Trap_T103,Trap_T107,Trap_T114,Trap_T115,Trap_T128,Trap_T129,Trap_T135,Trap_T138,Trap_T141,Trap_T142,Trap_T143,Trap_T144,Trap_T145,Trap_T146,Trap_T147,Trap_T148,Trap_T149,Trap_T150,Trap_T151,Trap_T152,Trap_T153,Trap_T154,Trap_T155,Trap_T156,Trap_T157,Trap_T158,Trap_T159,Trap_T160,Trap_T161,Trap_T162,Trap_T200,Trap_T206,Trap_T209,Trap_T212,Trap_T215,Trap_T218,Trap_T219,Trap_T220,Trap_T221,Trap_T222,Trap_T223,Trap_T224,Trap_T225,Trap_T226,Trap_T227,Trap_T228,Trap_T229,Trap_T230,Trap_T231,Trap_T232,Trap_T233,Trap_T235,Trap_T236,Trap_T237,Trap_T238,Trap_T900,Trap_T903
WnvPresent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,13764,94194,1,5367,5946,83,6,216,49930.648374,16113688,61535,124110,253579,14,132,99,12,12,20,23,104,88,114,93,88,24,28,51,77,63,22,24,83,104,101,134,88,16,69,21,26,76,2,61,14,27,106,107,135,82,16,15,122,8,14,93,78,101,97,102,24,76,15,17,11,119,105,18,5,31,6,76,87,38,82,66,22,28,36,25,86,128,28,23,114,5,83,39,14,45,22,107,28,20,117,128,112,28,139,120,21,16,23,67,78,29,94,67,14,22,137,90,38,18,76,19,14,116,100,84,23,19,99,21,125,117,32,70,21,90,85,59,66,53,73,71,66,53,8,56,35,47,40,54,31,3,7,125,87
1,723,40845,0,409,273,0,0,0,5169.351626,918631,3727,7134,15413,0,15,14,0,2,4,0,9,8,11,4,10,4,4,5,0,0,0,0,7,9,7,5,5,0,3,1,1,4,0,0,0,3,0,3,4,1,0,0,5,1,0,8,1,2,4,6,1,1,2,0,0,4,2,0,0,1,0,1,2,3,8,2,1,1,7,0,4,7,1,0,4,0,5,5,1,0,0,2,4,2,9,15,7,0,8,9,0,1,5,2,1,0,4,0,0,0,6,2,0,2,6,1,0,5,4,4,0,1,1,0,3,3,3,2,0,4,7,1,9,1,11,4,4,6,0,7,4,2,6,7,1,0,0,29,9


In [128]:
clean_train.to_csv("clean_train.csv")
clean_test.to_csv("clean_test.csv")

### 6.3 Cleaning Spray Dataset  <a id='clean spray 2'></a> 
<div align="right"><a href='#Table of Contents'>Back to Table of Contents</a></div>

In [129]:
def split_time(df):
    df.Date = pd.to_datetime(df.Date)
    df['am_pm'] = df['Time'].apply(lambda x: str(x).split()[-1])
    df['Time'] = df['Time'].apply(lambda x: str(x).split()[0])
    df['Time'] = pd.to_datetime(df['Time'], format = '%H:%M:%S').dt.time

In [130]:
split_time(df_spray_2)

In [131]:
df_spray_2.head()

,index,Date,Time,Latitude,Longitude,am_pm
0,0,2011-08-29,06:56:58,42.391623,-88.089163,PM
1,1,2011-08-29,06:57:08,42.391348,-88.089163,PM
2,2,2011-08-29,06:57:18,42.391022,-88.089157,PM
3,3,2011-08-29,06:57:28,42.390637,-88.089158,PM
4,4,2011-08-29,06:57:38,42.390410,-88.088858,PM


In [132]:
#merge weather data with train and test data
df_weather_2['Date'] = df_weather_2['Date'].astype(str)
clean_train['Date'] = clean_train['Date'].astype(str)
model2_train = clean_train.merge(df_weather_2, how = 'left', on = ['Date','Station'])
model2_test = clean_test.merge(df_weather_2, how = 'left', on = ['Date', 'Station'])

In [133]:
model2_train.to_csv("clean_train.csv")
model2_test.to_csv("clean_test.csv")